In [ ]:
!pip install groq==0.4.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
import os
from groq import Groq
import requests

In [ ]:
client = Groq(api_key="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")

In [ ]:
conversation_history = []
system_prompt = "You are a helpful assistant."

In [ ]:
def process_image(image_path):
    endpoint1 = "https://skin-classifier.tibot.ai/api/predict"
    endpoint2 = "https://skin-classifier.tibot.ai/api/predict_skip_healthy_check"

    with open(image_path, 'rb') as image_file:
        files = {'image_file': image_file}

        endpoints = [endpoint1, endpoint2]

        for endpoint in endpoints:
            try:
                response = requests.post(endpoint, files=files)

                if response.status_code == 200:
                    return response.json()
            except requests.exceptions.RequestException as e:
                print(f"Error processing image with {endpoint}: {e}")

    return {"error": "Failed to process image with both endpoints"}

def get_llama_response(prompt, conversation_history):
    messages = [
        {
            "role": "system",
            "content": system_prompt
        }
    ]

    for message in conversation_history:
        messages.append({
            "role": message["role"],
            "content": message["content"]
        })

    messages.append({
        "role": "user",
        "content": prompt
    })

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.1-8b-instant",
    )
    return chat_completion.choices[0].message.content



In [ ]:
def update_system_prompt(result):
    global system_prompt
    system_prompt += f"""
                                        <<SYS>>
    You are a medical chatbot.
    You help diagnose skin conditions based on the answers to the questions you will ask the user.
    You ask only one question at a time and wait for the human to respond with an answer to ask the next question.
    You do not make up user queries and have a conversation with yourself.

    You will try to narrow down which of the following conditions the user might have by asking the relevant questions:
    Acne & Rosacea: Acne is a disfiguring disorder of the pilosebaceous apparatus characterized by comedones, papules, pustules, cysts and scars.
    Bacterial Infections: Bacterial skin infections often begin as small, red bumps that slowly increase in size. Some bacterial infections are mild and easily treated with topical antibiotics, but other infections require an oral antibiotic.
    Benign Tumors: Skin tumors are abnormal growths of tissue. Skin tumors become extremely common as people get older.
    Eczema: Eczema is a condition where patches of skin become inflamed, itchy, red, cracked, and rough.
    Fungal Infections: Fungal infections, or mycosis, are diseases caused by a fungus (yeast or mold). Fungal infections are most common on skin or nails.
    Immunological Skin Disorder: Immunological Skin Disorder occur when the immune system cannot differentiate between healthy cells in the body and foreign substances or antigens – so it attacks them.
    Infestations, stings, and bites: Infestations with parasites can cause systemic diseases in humans, but often result in cutaneous lesions which are on the rise in dermatology clinical practice due to the increase of travel and immigration.
    Pigmentation Disorders: Pigmentation means coloring. Skin pigmentation disorders affect the color of the skin. Some pigmentation disorders affect just patches of skin. Others affect the entire body.
    Psoriasis: soriasis is a chronic, noncommunicable, painful, disfiguring and disabling disease for which there is no cure and with great negative impact on patients’ quality of life.
    Suspicious Tumors: Suspicious tumour maybe benign or may be malignant. After biopsy it will confirm. But when anyone suspecting a tumour, must consult with a doctor.
    Viral Infections: A viral infection is a proliferation of a harmful virus inside the body. Viral skin infections are a broad category encompassing a wide variety of conditions caused by a virus.

    You will ask the following questions in the order you think will help you get to the assessment in the shortest number of questions to assess the user's condition:

    Question) You ask the gender and age of the user.

    Question) You ask on which part of the body the user is having the problem.

    Question) You ask whether the user is experiencing any itching or not.

    Question) You will ask on a scale of 1 to 10, what is the scale of the itching they are experiencing:
    a) 1 - 2
    b) 3 - 5
    c) 6 - 8
    d) 9 - 10

    Question) You will ask the user whether any liquid is oozing.

    Question) You will ask the user since when they have been having the problem.

    Question) you will ask the user if there is any other condition they are experiencing.
    If the answer is yes then you will ask follow up questions to assess



    You will take into consideration the scores in {result} and ask questions to confirm which skin condition the user has.
    For Example:
    If the higest score is greater than 90% and it is Eczema, you will ask questions that are likely to be asked by a doctor for an Eczema patient.
    If the higest score is greater than 90% and it is Psoriasis, you will ask questions that are likely to be asked by a doctor for a Psoriasis patient.
    But if the higest score is less than 90%, then you will consider the top 3 scores and their skin conditions and ask questions to narrow down which of the top 3 skin conditions the user might have.

    After answering of quetions, if the probability of having a certain skin condition goes up then you will display it to the user and if the probability goes down, you will also display it. You will assess this based on the answers the user provides

    **Perform Differential Diagnosis:**After you are done asking the final question, you will take into account the initial scores in {result} and the answers of the questions the user has provided and then perform a differential diagnosis by making a list of the possible skin conditions the user might have that are causing the symptoms they have described.

    If you are satisfied with the answers and can assess their skin condition after doing differential dignosis then you will end the conversation and tell them your assessment. If not then you will ask follow up questions to reach a conclusion about their skin condition and then tell them your assessment.


    **Provide reasoning for justification of final assessment:** After concluding your assessment of the user's skin condition, you will provide detailed justification and explanations for your reasoning.

    Before the conversation starts, display the text provided in {result} variable also show which skin condition has the higest probability score.
    <</SYS>>
                    """

In [ ]:
print("You can start chatting or type 'upload' to process an image.")
print("Type 'end' to finish the conversation.")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() == "end":
        print("Bot: Chat ended.")
        break

    if user_input.lower() == "upload":
        image_path = input("Enter the path to your image file: ").strip()
        if os.path.exists(image_path):
            results = process_image(image_path)
            update_system_prompt(results)
            print("Image Processing Results:", results['predictions'])
        else:
            print("Error: File not found. Please provide a valid file path.")
        continue

    bot_response = get_llama_response(user_input, conversation_history)

    conversation_history.append({"role": "user", "content": user_input})
    conversation_history.append({"role": "assistant", "content": bot_response})

    print(f"Bot: {bot_response}")

You can start chatting or type 'upload' to process an image.
Type 'end' to finish the conversation.
You: upload
Enter the path to your image file: /content/97.jpg
Image Processing Results: [{'label': 'Eczema', 'probability': '89.78%'}, {'label': 'Pigmentation Disorder', 'probability': '10.22%'}, {'label': 'Benign Tumor', 'probability': '0.00%'}, {'label': 'Malignant Tumor', 'probability': '0.00%'}, {'label': 'Psoriasis', 'probability': '0.00%'}]
You: start
Bot: **Initial Diagnosis Scores:**
{'status': 'ok', 'predictions': [{'label': 'Eczema', 'probability': '89.78%'}, {'label': 'Pigmentation Disorder', 'probability': '10.22%'}, {'label': 'Benign Tumor', 'probability': '0.00%'}, {'label': 'Malignant Tumor', 'probability': '0.00%'}, {'label': 'Psoriasis', 'probability': '0.00%'}]}

**Condition with Highest Probability:**
Eczema (89.78%)

To start, can you please provide the following information:

**1. Your age:** _______________________________________
**2. Your gender:** ______________

In [ ]:
conversation_history

[{'role': 'user', 'content': 'start'},
 {'role': 'assistant',
  'content': "**Initial Diagnosis Scores:**\n{'status': 'ok', 'predictions': [{'label': 'Eczema', 'probability': '89.78%'}, {'label': 'Pigmentation Disorder', 'probability': '10.22%'}, {'label': 'Benign Tumor', 'probability': '0.00%'}, {'label': 'Malignant Tumor', 'probability': '0.00%'}, {'label': 'Psoriasis', 'probability': '0.00%'}]}\n\n**Condition with Highest Probability:**\nEczema (89.78%)\n\nTo start, can you please provide the following information:\n\n**1. Your age:** _______________________________________\n**2. Your gender:** _______________________________________"},
 {'role': 'user', 'content': '31 male'},
 {'role': 'assistant',
  'content': '**Age:** 31\n**Gender:** Male\n\n**Next Question:**\nOn which part of your body are you experiencing the problem?\n\n(Please specify the location, e.g., face, hands, arms, legs, torso, etc.)'},
 {'role': 'user', 'content': 'legs'},
 {'role': 'assistant',
  'content': '**Loc

In [ ]:
results

{'status': 'ok',
 'predictions': [{'label': 'Eczema', 'probability': '89.78%'},
  {'label': 'Pigmentation Disorder', 'probability': '10.22%'},
  {'label': 'Benign Tumor', 'probability': '0.00%'},
  {'label': 'Malignant Tumor', 'probability': '0.00%'},
  {'label': 'Psoriasis', 'probability': '0.00%'}]}

In [ ]:
# messages = [
#     {"role": "system", "content": """You are a helpful medical chatbot that helps with diagnosing skin condition."""},
#     {
#         "role": "user",
#         "content": "Give me the top three skin condition scores"
#     },

# ]
# tools = [
#     {
#         "type": "function",
#         "function": {
#             "name": "process_image",
#             "description": "Get skin condition prediction scores from processing the image",
#             "parameters": {
#                 "type": "object",
#                 "properties": {

#                 },
#                 "required": []
#             }
#         }
#     }
# ]

In [ ]:
# try:
#     response_1 = client.chat.completions.create(
#         model="llama3-groq-70b-8192-tool-use-preview",
#         messages=messages,
#         tools=tools,
#         tool_choice="auto",
#         max_tokens=4096
#     )
#     print(response_1.choices[0].message.content)
# except Exception as e:
#     print(f"Error: {str(e)}")

In [ ]:
import json
def get_llm_agent_assessment(image_scores, conversation_history):
    system_prompt = """
    You are an advanced medical AI agent specializing in dermatology. Your task is to reassess the diagnosis made by a chatbot based on image processing scores and the entire conversation history with the user. You will:
    1. Review the image processing scores and the full conversation history.
    2. Identify all relevant answers given by the user throughout the conversation.
    3. Perform a differential diagnosis based on this information.
    4. Confirm or revise the initial diagnosis made by the chatbot.
    5. Critique the initial diagnosis made by the chatbot based on the user's answers thoroughly.
    6. Provide a detailed justification for your final assessment, including why you agree or disagree with the initial diagnosis.
    7. Act as the skin specialist who provided a second opinion and revies and reassesses the initila assessment made by the chatbot.

    Please structure your response as follows:
    1. Summary of User Information
    2. Review of Initial Diagnosis
    3. Differential Diagnosis
    4. Crique of the Initial Diagnosis
    4. Final Assessment
    5. Detailed Justification
    """

    user_prompt = f"""
    Image processing scores: {json.dumps(image_scores)}

    Conversation history:
    {json.dumps(conversation_history)}

    Please provide your reassessment based on the above information.
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content

In [ ]:
final_assessment = get_llm_agent_assessment(results, conversation_history)


In [ ]:
print(final_assessment)

**1. Summary of User Information**

The user, a 31-year-old male, initially consulted a chatbot about a skin condition on his legs. Based on the user's input, the chatbot determined the following information:

- Age: 31
- Gender: Male
- Location of skin condition: Legs
- Itching or discomfort: Yes
- Severity of itching: Moderate (6-8)
- Oozing or discharge: No
- Duration of symptoms: A few days
- Additional symptoms: Redness around the affected area
- Exposure to allergens or irritants: Not really

**2. Review of Initial Diagnosis**

The chatbot provided an initial diagnosis based on image processing scores, which indicated a 89.78% probability of eczema, 10.22% probability of pigmentation disorder, and 0.00% probability of other conditions. After gathering more information from the user, the chatbot updated the diagnosis scores, ultimately concluding that the user likely had eczema (94.78% probability) due to the presence of redness, itching, and moderate severity of symptoms.

**3. D

##Crewai agentic flow

In [ ]:
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
#Updated versions
!pip install crewai_tools==0.4.8
!pip install crewai==0.41.1
!pip install langchain_groq==0.1.6

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mem0ai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to d

In [ ]:
import os
os.environ["OPENAI_API_KEY"]="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"
os.environ["OPENAI_MODEL_NAME"]="llama-3.1-70b-versatile"
os.environ["OPENAI_API_BASE"]= "https://api.groq.com/openai/v1"

In [ ]:
os.environ["OPENAI_EMBEDDING_MODEL"] = ""

In [ ]:
!pip install python-dotenv

In [ ]:
# !pip install pydantic==1.10.13
# !pip install langchain==0.0.329
# !pip install crewai

  Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (149 kB)
Using cached pydantic-1.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.122 requires langchain<=0.3,>0.2, which is not installed.
albumentations 1.4.15 requires pydantic>=2.7.0, but you have pydantic 1.10.13 which is incompatible.
instructor 1.3.3 requires pydantic<3.0.0,>=2.7.0, but you have pydantic 1.10.13 which is incompatible.
langchain-groq 0.2.0 requires langchain-core<0.4,>=0.3, but you have langchain-core 0.2.41 which is incompatible.
litellm 1.48.9 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.13 which is incompatible.
mem0ai 0.1.17 requires pydantic<3.0.0,>=2.7.3, but you have pydantic 1.10.13 which is incompatible.


  Using cached langchain-0.0.329-py3-none-any.whl.metadata (16 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.329-py3-none-any.whl (2.0 MB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.129
    Uninstalling langsmith-0.1.129:
      Successfully uninstalled langsmith-0.1.129
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.122 requires langchain<=0.3,>0.2, but you have langchain 0.0.329 which is incompatible.
langchain-community 0.2.17 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.0.329 which is incompatible.
langchain-community 0.2.17 requires langsmith<0.2.0,>=0.1.112, but you have langsmith 0.0.92 which is incompatible.
langchain-core 0.2.41 requires langsmith<0.2.0,>=0.1.112, but you have lan

In [ ]:
from crewai import Agent

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
# !pip install -qqq langchain==0.1.6 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.61.0 requires langchain<0.3.0,>=0.2.16, but you have langchain 0.1.6 which is incompatible.
crewai 0.61.0 requires pydantic<3.0.0,>=2.4.2, but you have pydantic 2.0.3 which is incompatible.
embedchain 0.1.121 requires langchain<=0.3,>0.2, but you have langchain 0.1.6 which is incompatible.
embedchain 0.1.121 requires langchain-community<0.3.0,>=0.2.6, but you have langchain-community 0.0.20 which is incompatible.
langchain-cohere 0.1.9 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.1.23 which is incompatible.
langchain-experimental 0.0.65 requires langchain-community<0.3.0,>=0.2.16, but you have langchain-community 0.0.20 which is incompatible.
langchain-experimental 0.0.65 requires langchain-core<0.3.0,>=0.2.38, but you have langchain-core 0.1.23 which is incompatible.
langchain-groq 

In [ ]:
import os

In [ ]:
# !pip install langchain==0.1.0
# !pip install langchain-groq==0.1.5
# !pip install groq==0.4.2

  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide

In [ ]:
from langchain_groq import ChatGroq

os.environ['groq_api_key']='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'
groq_chat=ChatGroq(groq_api_key=os.environ['groq_api_key'],
             model_name="llama-3.1-70b-versatile",
             temperature=0,
             max_tokens=7800,


                   )






In [ ]:
# from langchain.chat_models import ChatOpenAI

In [ ]:
# os.environ["OPENAI_API_KEY"] = "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"
# os.environ["OPENAI_API_BASE"] = "https://api.groq.com/openai/v1"

# # Initialize chat model
# llm = ChatOpenAI(
#     api_key=os.environ["OPENAI_API_KEY"],
#     base_url=os.environ["OPENAI_API_BASE"],
#     model_name="llama-3.1-70b-versatile",  # Using a model supported by Groq
#     temperature=0,
#     max_tokens=7800,

# )

<ipython-input-73-c8cf1f065bf9>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [ ]:
specialist_1 = Agent(
    role='Skin specialist who reviews the image processing scores and the entire conversation history with  the user',
    goal="""Comprehensively reassess and critique the diagnosis made by the chatbot based on image processing scores and the entire conversation history with the user.
    Go through every answer the user has provided to the chatbot questions and critically assess them to confirm the diagnosis and find out if there was any inconsistencies.
    After carefully analyzing the image scores and the user's answers, provide your own assessment of the skin condition the user might have.

    """,
    verbose=True,

    backstory=(
       """Expert in skin diseases. Can succesfully identify skin diseases based on patient history and the conversation with the chatbot.

       """
    ),
    llm=groq_chat,
    max_iter=2,
    allow_delegation=True



)

specialist_2 = Agent(
    role='Skin disease specialist',
    goal="""
    Review the diagnosis from specialist_1 and give your final assessment along with the possible paths the user can take for treatment and remedying.
    Be gentle while giving your personalized feedback.

    """,
    verbose=True,

    backstory=(
        """
        Skin disease expert who assesses the review and assessment from specialist_1 and gives the final assessment and takes the patient through the steps of treatment to get better.

        """
    ),
    llm=groq_chat,
    max_iter=2,
    allow_delegation=False



)

In [ ]:
from crewai import Task

sp_task1 = Task(
    description=(
        f"Get the image processing scores {results} and entire conversation history {conversation_history} "
    ),
    expected_output="""
    Please structure your response as follows:
        1. Summary of User Information
        2. Review of Initial Diagnosis
        3. Differential Diagnosis
        4. Crique of the Initial Diagnosis
        4. Final Assessment
        5. Detailed Justification
    """,

    agent=specialist_1,
)

sp_task2 = Task(
    description=(
        f"Comprehensively review and go through the final assessment with the image processing scores {results} and entire conversation history {conversation_history} and suggest treatment options and cater to whatever the skin disease the user might have."
    ),
    expected_output="""
    Structure your response in the following manner:
        1. The review of the specialist_1
        2. The final assessment
        3. Possible treatment options
        4. Possible remedy options

    **Present your response in the exact form a doctor would in their report**

    """,

    agent=specialist_2,
    async_execution=False,
)

In [ ]:
from crewai import Crew, Process

crew = Crew(
  agents=[specialist_1, specialist_2],
  tasks=[sp_task1, sp_task2],
  process=Process.sequential,

  max_rpm=29,



)

assessment = crew.kickoff()
print(assessment)



> Entering new CrewAgentExecutor chain...
Thought: I need to review the conversation history and image processing scores to confirm the diagnosis made by the chatbot.

Action: Delegate work to coworker
Action Input: {"task": "Review conversation history and image processing scores", "context": "Conversation history: [{'role': 'user', 'content': 'start'}, {'role': 'assistant', 'content': \"**Initial Diagnosis Scores:**\\n{'status': 'ok', 'predictions': [{'label': 'Eczema', 'probability': '89.78%'}, {'label': 'Pigmentation Disorder', 'probability': '10.22%'}, {'label': 'Benign Tumor', 'probability': '0.00%'}, {'label': 'Malignant Tumor', 'probability': '0.00%'}, {'label': 'Psoriasis', 'probability': '0.00%'}]}\", 'role': 'assistant', 'content': \"**Condition with Highest Probability:**\\nEczema (89.78%)\\n\\nTo start, can you please provide the following information:\\n\\n**1. Your age:** _______________________________________\\n**2. Your gender:** ____________________________________